In [16]:
# Basic Libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

#Evaluation library
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Importing the model definition and data loading functions from .py files
from fashion_mnist_3layer_NN_model import load_and_preprocess_data_3, create_model_3_layer
from fashion_mnist_6layer_NN_model import load_and_preprocess_data_6, create_model_6_layer
from fashion_mnist_12layer_NN_model import load_and_preprocess_data_12, create_model_12_layer
from utils import class_names

In [2]:
# Load and preprocess the data
train_images, train_labels, test_images, test_labels = load_and_preprocess_data_3()

In [3]:
# Create the model
model = create_model_3_layer()

In [ ]:
# Compile the model with a chosen optimizer

In [18]:
# Train the model

In [19]:
# Evaluate the model

In [20]:
# Predict the model